In [1]:
%load_ext autoreload
%autoreload 2

In [25]:
import jax.numpy as jnp
from beartype import beartype as typechecker
from jaxtyping import Array,Float, jaxtyped, PyTree
import equinox as eqx
import jax
from nn_magnetics.jax.train_isotropic import L1loss
from typing import List

In [29]:
class Model(eqx.Module):
    layers: List

    def __init__(self, key):
        key1, key2, key3, key4, key5 = jax.random.split(key, 5)
        self.layers = [
            eqx.nn.Linear(in_features=6, out_features=48, key=key1),
            jax.nn.silu,
            eqx.nn.Linear(in_features=48, out_features=24, key=key2),
            jax.nn.silu,
            eqx.nn.Linear(in_features=24, out_features=12, key=key3),
            jax.nn.silu,
            eqx.nn.Linear(in_features=12, out_features=6, key=key4),
            jax.nn.silu,
            eqx.nn.Linear(in_features=6, out_features=3, key=key5),
        ]
        
    @jaxtyped(typechecker=typechecker)
    def __call__(self, x: Float[Array, "batch 6"]) -> Float[Array, "batch 3"]:
        for layer in self.layers:
            x = layer(x)

        return x

In [30]:
model = Model(jax.random.PRNGKey(1))

In [ ]:
@jaxtyped(typechecker=typechecker)
def loss_fn(
    model: Model,
    X: Float[Array, "batch 6"],
    B: Float[Array, "batch 6"],
) -> Float:
    B_demag, B_ana = B[..., :3], B[..., 3:]
    B_corrections = jax.vmap(model)(X)

    B_pred = B_ana * B_corrections

    return L1loss(y=B_demag, y_pred=B_pred)

BeartypeDecorHintNonpepException: Function __main__.loss_fn() return <class 'jaxtyping.Float'> uncheckable at runtime (i.e., not passable as second parameter to isinstance(), due to raising "AnnotationError: Do not use `isinstance(x, jaxtyping.Float)`. If you want to check just the dtype of an array, then use `jaxtyping.Float[jnp.ndarray, "..."]`." from metaclass _MetaAbstractDtype.__instancecheck__() method).

In [18]:
a = jnp.ones((10, 6))
b = jnp.ones((10, 6))

In [23]:
loss = eqx.filter_value_and_grad(loss_fn)(model, a, b)

TypeError: Gradient only defined for scalar-output functions. Output was (Array(1.0289239, dtype=float32), Model(
  layers=[
    Linear(
      weight=f32[48,6],
      bias=f32[48],
      in_features=6,
      out_features=48,
      use_bias=True
    ),
    None,
    Linear(
      weight=f32[24,48],
      bias=f32[24],
      in_features=48,
      out_features=24,
      use_bias=True
    ),
    None,
    Linear(
      weight=f32[12,24],
      bias=f32[12],
      in_features=24,
      out_features=12,
      use_bias=True
    ),
    None,
    Linear(
      weight=f32[6,12],
      bias=f32[6],
      in_features=12,
      out_features=6,
      use_bias=True
    ),
    None,
    Linear(
      weight=f32[3,6],
      bias=f32[3],
      in_features=6,
      out_features=3,
      use_bias=True
    )
  ]
)).

In [11]:
loss

Array(0., dtype=float32)